In [ ]:
!pip install sacremoses
!pip install flair

In [ ]:
!!pip install textattack['tensorflow']

['Requirement already satisfied: textattack[tensorflow] in /usr/local/lib/python3.7/dist-packages (0.3.4)',
 'Requirement already satisfied: num2words in /usr/local/lib/python3.7/dist-packages (from textattack[tensorflow]) (0.5.10)',
 'Requirement already satisfied: torch!=1.8,>=1.7.0 in /usr/local/lib/python3.7/dist-packages (from textattack[tensorflow]) (1.11.0+cu113)',
 'Requirement already satisfied: nltk in /usr/local/lib/python3.7/dist-packages (from textattack[tensorflow]) (3.2.5)',
 'Requirement already satisfied: scipy>=1.4.1 in /usr/local/lib/python3.7/dist-packages (from textattack[tensorflow]) (1.4.1)',
 'Requirement already satisfied: flair in /usr/local/lib/python3.7/dist-packages (from textattack[tensorflow]) (0.11.2)',
 'Requirement already satisfied: lru-dict in /usr/local/lib/python3.7/dist-packages (from textattack[tensorflow]) (1.1.7)',
 'Requirement already satisfied: terminaltables in /usr/local/lib/python3.7/dist-packages (from textattack[tensorflow]) (3.1.10)',


In [ ]:
import textattack

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:12<00:00, 38.8MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpq53n3skt.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [ ]:
import pickle

In [ ]:
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

MODEL = f'cardiffnlp/twitter-roberta-base-sentiment-latest'
test_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
test_tokenizer = AutoTokenizer.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# Folder to mount
FOLDERNAME = 'cs329t/project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Ensure the Python interpreter of the Colab VM can load python files
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# cd into the dataset
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/cs329t/project


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import csv

# Load CSV file
def load_csv(file_path):
  with open(file_path, newline='') as csv_file:
    reader = csv.reader(csv_file)
    csv_data = list(reader)
  return csv_data

csv_data = load_csv('data/target_test_clean.csv')

In [ ]:
def format_data(csv_data):
  formatted_dataset = list()
  for example in csv_data[1:]:
    text = example[0]
    targets = [word for word in text.split() if word.startswith('@') or word.startswith('.@')]
    polarity = example[-1]
    formatted_dataset.append([text, targets, polarity])
  return formatted_dataset
  
dataset = format_data(csv_data)

In [ ]:
def neutralize_dataset(dataset):
  neutralized_dataset = list()
  for example in dataset:
    text = example[0]
    targets = example[1]
    polarity = example[-1]
    for target in targets:
      text = text.replace(target, '@username')
    neutralized_dataset.append([text, targets, polarity])
  neutralized_dataset = [example for example in neutralized_dataset if '@username' in example[0]]
  return neutralized_dataset

neutralized_dataset = neutralize_dataset(dataset)

In [ ]:
def parse_label(res):
    orig_label = int(res[0])
    if res[-10:] == ' [SKIPPED]':
        return orig_label, -1
    if res[-9:] == ' [FAILED]':
        return orig_label,orig_label
    else:
        return orig_label, res.split(">")[1][1]

def tuplify_dataset(dataset):
    return [(tweet,int(label)) for tweet,target,label in dataset]

In [ ]:
len(neutralized_dataset)

4343

In [ ]:
ta_model_wrapper = HuggingFaceModelWrapper(test_model,test_tokenizer)
attack_tf = TextFoolerJin2019.build(ta_model_wrapper)
attacker_tf = Attacker(attack_tf, textattack.datasets.Dataset(tuplify_dataset(neutralized_dataset[:100])))
attacker_tf.attack_args.num_examples = -1
attack_tf_results = attacker_tf.attack_dataset()
a_tf_res_list = [(res.original_text(), res.perturbed_text(), parse_label(res.goal_function_result_str())) for res in attack_tf_results]

with open('a_textfooler.pkl', 'wb') as f:
    pickle.dump(a_tf_res_list, f)

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       



  1%|          | 1/100 [04:50<7:59:56, 290.88s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 1/100 [04:50<7:59:57, 290.89s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (91%)]] --> [[[FAILED]]]

#NowPlaying on Activ Radio 90 FM : Kaleo - Way down we go (TOP24H #charts : 4th) @username @username #Music… 






[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   2%|▏         | 2/100 [07:23<6:02:15, 221.79s/it]

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   2%|▏         | 2/100 [07:23<6:02:15, 221.79s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (95%)]] --> [[[FAILED]]]

LPW "Mayhem" on Saturday April 22 @username Elks Hall (128 School St) Clinton Ma  






[Succeeded / Failed / Skipped / Total] 64 / 4 / 30 / 98:  99%|█████████▉| 99/100 [3:05:46<01:52, 112.59s/it]

[Succeeded / Failed / Skipped / Total] 65 / 4 / 30 / 99:  99%|█████████▉| 99/100 [3:05:46<01:52, 112.59s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (94%)]] --> [[1 (51%)]]

RT @[[username]] If you're [[miserable]] don't [[come]] on my [[page]]. You are not [[wanted]] in my [[comment]] [[section]]. Don't [[bring]] your [[funky]] [[ass]] [[attitude]] [[arou]]…

RT @[[user]] If you're [[dismal]] don't [[happens]] on my [[pages]]. You are not [[longing]] in my [[observation]] [[part]]. Don't [[porter]] your [[marvellous]] [[hind]] [[position]] [[somethi]]…






[Succeeded / Failed / Skipped / Total] 65 / 4 / 30 / 99: 100%|██████████| 100/100 [3:05:48<00:00, 111.49s/it]

[Succeeded / Failed / Skipped / Total] 65 / 4 / 31 / 100: 100%|██████████| 100/100 [3:05:48<00:00, 111.49s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[0 (70%)]] --> [[[SKIPPED]]]

@username So how you argue in favo of Outing being fit to have nuclear codes while supporting chemical murderer… 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 65     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 69.0%  |
| Accuracy under attack:        | 4.0%   |
| Attack success rate:          | 94.2%  |
| Average perturbed word %:     | 17.18% |
| Average num. words per input: | 16.54  |
| Avg num queries:              | 98.38  |
+-------------------------------+--------+


In [ ]:
ta_model_wrapper = HuggingFaceModelWrapper(test_model,test_tokenizer)
attack_dwb = textattack.attack_recipes.deepwordbug_gao_2018.DeepWordBugGao2018.build(ta_model_wrapper)
attacker_dwb = Attacker(attack_dwb, textattack.datasets.Dataset(tuplify_dataset(neutralized_dataset[:100])))
attacker_dwb.attack_args.num_examples = -1
attack_results_dwb = attacker_dwb.attack_dataset()

a_dwb_res__list = [(res.original_text(), res.perturbed_text(), parse_label(res.goal_function_result_str())) for res in attack_results_dwb]
with open('a_deepwordbug_100.pkl', 'wb') as f:
    pickle.dump(a_dwb_res__list, f)

textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 1/100 [02:07<3:29:34, 127.01s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (91%)]] --> [[[FAILED]]]

#NowPlaying on Activ Radio 90 FM : Kaleo - Way down we go (TOP24H #charts : 4th) @username @username #Music… 




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   2%|▏         | 2/100 [03:54<3:11:33, 117.28s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (95%)]] --> [[[FAILED]]]

LPW "Mayhem" on Saturday April 22 @username Elks Hall (128 School St) Clinton Ma  




[Succeeded / Failed / Skipped / Total] 44 / 25 / 30 / 99:  99%|█████████▉| 99/100 [1:21:42<00:49, 49.52s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (94%)]] --> [[[FAILED]]]

RT @username If you're miserable don't come on my page. You are not wanted in my comment section. Don't bring your funky ass attitude arou…




[Succeeded / Failed / Skipped / Total] 44 / 25 / 31 / 100: 100%|██████████| 100/100 [1:21:44<00:00, 49.04s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[0 (70%)]] --> [[[SKIPPED]]]

@username So how you argue in favo of Outing being fit to have nuclear codes while supporting chemical murderer… 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 44     |
| Number of failed attacks:     | 25     |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 69.0%  |
| Accuracy under attack:        | 25.0%  |
| Attack success rate:          | 63.77% |
| Average perturbed word %:     | 22.18% |
| Average num. words per input: | 16.54  |
| Avg num queries:              | 41.46  |
+-------------------------------+--------+


In [ ]:
a_dwb_res__list = [(res.original_text(), res.perturbed_text(), parse_label(res.goal_function_result_str())) for res in attack_results]
with open('a_deepwordbug.pkl', 'wb') as f:
    pickle.dump(a_dwb_res__list, f)

NameError: ignored

In [ ]:
ta_model_wrapper = HuggingFaceModelWrapper(test_model,test_tokenizer)
attack_ba = textattack.attack_recipes.bert_attack_li_2020.BERTAttackLi2020.build(ta_model_wrapper)
attacker_ba = Attacker(attack_ba, textattack.datasets.Dataset(tuplify_dataset(neutralized_dataset[:100])))
attacker_ba.attack_args.num_examples = -1
attack_results_ba = attacker_ba.attack_dataset()

a_ba_res__list = [(res.original_text(), res.perturbed_text(), parse_label(res.goal_function_result_str())) for res in attack_results_ba]
with open('a_bertattack.pkl', 'wb') as f:
    pickle.dump(a_ba_res__list, f)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  48
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.4
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [08:03<13:18:08, 483.73s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (91%)]] --> [[2 (71%)]]

#[[NowPlaying]] on Activ Radio 90 FM : [[Kaleo]] - [[Way]] down we go (TOP24H #charts : 4th) @username @username #[[Music]]… 

#[[nowplayy]] on Activ Radio 90 FM : [[kaleah]] - [[hands]] down we go (TOP24H #charts : 4th) @username @username #[[best]]… 




...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (95%)]] --> [[2 (51%)]]

LPW "Mayhem" on [[Saturday]] April 22 @username [[Elks]] Hall (128 School St) [[Clinton]] [[Ma]]  

LPW "Mayhem" on [[fun]] April 22 @username [[elks]] Hall (128 School St) [[green]] [[mass]]  




[Succeeded / Failed / Skipped / Total] 68 / 1 / 30 / 99:  99%|█████████▉| 99/100 [3:37:02<02:11, 131.55s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (94%)]] --> [[1 (54%)]]

RT @username If you're [[miserable]] don't come on my page. You are not [[wanted]] in my comment section. Don't bring your funky [[ass]] attitude arou…

RT @username If you're [[happy]] don't come on my page. You are not [[unwanted]] in my comment section. Don't bring your funky [[for]] attitude arou…




[Succeeded / Failed / Skipped / Total] 68 / 1 / 31 / 100: 100%|██████████| 100/100 [3:37:05<00:00, 130.25s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[0 (70%)]] --> [[[SKIPPED]]]

@username So how you argue in favo of Outing being fit to have nuclear codes while supporting chemical murderer… 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 68     |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 69.0%  |
| Accuracy under attack:        | 1.0%   |
| Attack success rate:          | 98.55% |
| Average perturbed word %:     | 13.53% |
| Average num. words per input: | 16.54  |
| Avg num queries:              | 103.77 |
+-------------------------------+--------+


In [ ]:
attacker_tf.attack_args.num_examples = -1

In [ ]:
attacker_tf.attack_args.num_examples

100